<a href="https://colab.research.google.com/github/ebelleng/EDDA_desafio1/blob/main/Copy_of_Packing_squares_algoritms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Packing Squares 


In [ ]:
def hola_mundo():
  print("Hola mundo")

In [ ]:
hola_mundo()

Hola mundo


In [1]:
lista = [1,2,3]
lista

[1, 2, 3]